# 가설 3: 오류 주석 활용 검색 정확도 개선

## 배경
- 가설 2에서 BM25/벡터/형태소 N-gram을 조합하여 유사 오류 사례를 검색했으나, 정확도가 낮았음
- 근본 원인: 현재 시스템은 **"유사 문장"**을 찾을 뿐, **"유사 오류"**를 찾지 못함
- 예) "친구**하고** 약속 있어요" (하고→와, 조사 대치)와 "동생**하고** 밥 먹었어요" (하고→와, 동일 조사 대치)는 내용이 다르지만 오류 패턴이 동일함

## 현재 OpenSearch 인덱스 (`korean_test`)
| 필드 | 타입 | 역할 |
|------|------|------|
| `original_text` | text (nori) | BM25 키워드 검색 |
| `morphs` | keyword[] | 품사 태그 배열 |
| `embedding` | knn_vector (1024d) | 벡터 유사도 검색 |

→ 문장 수준 정보만 존재, **오류 정보 없음**

## 말뭉치에 존재하지만 미활용 중인 데이터

| 컬럼 | 설명 | 예시 |
|------|------|------|
| `오류 위치` | 오류 발생 형태소 위치 코드 | CNNG, FAP, FOP, FED, FNP |
| `오류 양상` | 오류 유형 | REP(대치), MIF(오형), OM(누락), ADD(첨가) |
| `오류 층위` | 오류의 언어학적 층위 | PP(표기), MCJ(형태소결합), DS(방언), ST(문체) |
| `원 형태소` / `형태 주석` | 오류 형태소와 품사 | 하고 / JKB |
| `교정 형태소` / `교정 주석` | 교정된 형태소와 품사 | 와 / JKB |

오류 주석이 있는 문장: **65,791개** / 전체 235,902개 (약 28%)

## 가설

### 핵심 가설
> 오류 주석 데이터(오류 위치, 오류 양상, 오류 층위, 교정 쌍)를 OpenSearch 인덱스에 구조화하여 추가하면,  
> 입력 문장과 **동일한 유형의 오류**를 가진 말뭉치 사례를 더 정확하게 검색할 수 있다.

### 세부 가설

**가설 3-1: 오류 시그니처 인덱싱**
- 문장별 오류를 `오류위치:오류양상:오류층위` 형태의 시그니처로 변환하여 keyword 필드로 인덱싱
- 예) `["FAP:REP:DS", "FNP:OM"]`
- term query로 **구조적으로 동일한 오류 패턴**을 직접 검색 가능

**가설 3-2: 교정 쌍 인덱싱**
- `원형태소(품사)→교정형태소(품사)` 쌍을 인덱싱
- 예) `["하고(JKB)→와(JKB)", "0→이(JKS)"]`
- 빈번한 오류 패턴(조사 혼동 등)의 **구체적 매칭**에 효과적

**가설 3-3: 2단계 검색 (Two-Stage Retrieval)**
- Stage 1: 기존 BM25 + 벡터 검색으로 후보 확보
- Stage 2: 후보 문장들의 오류 주석을 분석하여 입력의 오류 유형 역추정 → 오류 시그니처로 2차 검색
- 입력 문장에 오류 주석이 없는 문제를 **후보 기반 역추정**으로 우회

## 검증 계획

### Step 1. 데이터 준비
- 말뭉치에서 오류 주석을 문장 단위로 집계
- 오류 시그니처 / 교정 쌍 생성

### Step 2. OpenSearch 인덱스 확장
- 기존 `korean_test` 인덱스에 오류 관련 필드 추가 (또는 새 인덱스 생성)
- 추가 필드: `error_signatures` (keyword[]), `correction_pairs` (keyword[]), `error_patterns` (keyword[]) 등

### Step 3. 검색 파이프라인 구현
- 3-1: 오류 시그니처 기반 term 검색
- 3-2: 교정 쌍 기반 term 검색
- 3-3: 기존 검색 → 오류 역추정 → 2차 검색

### Step 4. 평가
- 가설 2와 동일한 테스트 문장으로 비교
- 검색된 결과가 실제로 "유사한 오류"인지 정성 평가

---
## 구현 시작

### Step 1. 말뭉치 오류 주석 집계

In [1]:
import pandas as pd

df = pd.read_parquet('말뭉치.parquet.gzip')
print(f'전체 형태소 행: {len(df):,}')
print(f'전체 문장 수: {df.groupby(["표본 번호","문장"]).ngroups:,}')

전체 형태소 행: 2,608,912
전체 문장 수: 235,902


In [2]:
# 오류가 있는 행만 필터
err_rows = df[df['오류 양상'] != '0'].copy()
print(f'오류 행 수: {len(err_rows):,}')

# 오류 시그니처 생성: 오류위치:오류양상:오류층위
def make_error_signature(row):
    loc = row['오류 위치']
    pat = row['오류 양상']
    lvl = row['오류 층위'] if row['오류 층위'] != '0' else ''
    if lvl:
        return f"{loc}:{pat}:{lvl}"
    return f"{loc}:{pat}"

err_rows['signature'] = err_rows.apply(make_error_signature, axis=1)

# 교정 쌍 생성: 원형태소(품사)→교정형태소(품사)
def make_correction_pair(row):
    orig = row['원 형태소'] if row['원 형태소'] != '0' else '∅'
    orig_tag = row['형태 주석'] if row['형태 주석'] != '0' else ''
    corr = row['교정 형태소'] if row['교정 형태소'] != '0' else '∅'
    corr_tag = row['교정 주석'] if row['교정 주석'] != '0' else ''
    
    orig_str = f"{orig}/{orig_tag}" if orig_tag else orig
    corr_str = f"{corr}/{corr_tag}" if corr_tag else corr
    return f"{orig_str}→{corr_str}"

err_rows['correction_pair'] = err_rows.apply(make_correction_pair, axis=1)

print()
print('=== 오류 시그니처 샘플 ===')
print(err_rows[['문장', 'signature', 'correction_pair']].head(10).to_string())

오류 행 수: 137,298

=== 오류 시그니처 샘플 ===
                             문장    signature  correction_pair
37       수업이 끝난 후에 친구하고 약속 있어요.   FAP:REP:DS     하고/JKB→와/JKB
39       수업이 끝난 후에 친구하고 약속 있어요.       FNP:OM          ∅→이/JKS
84   혼자 집에서 밥 먹고 텔레비전 보고 재미있어요.      FOP:REP          ∅→을/JKO
88   혼자 집에서 밥 먹고 텔레비전 보고 재미있어요.       FOP:OM          ∅→을/JKO
90   혼자 집에서 밥 먹고 텔레비전 보고 재미있어요.      FED:REP       고/EC→아서/EC
103      그래서 열두 시에 다 끝난면 자도 돼요.  CMAJ:REP:DC  그래서/MAJ→그리고/MAJ
106      그래서 열두 시에 다 끝난면 자도 돼요.      CNNG:OM          ∅→전/NNG
111      그래서 열두 시에 다 끝난면 자도 돼요.  FED:MIF:MCJ        면/EC→면/EC
162             친구들을 이야기하고 싶어요.      FOP:REP      을/JKO→과/JKB
174   그래서 토요일 저녁에 친구하고 술을 마셨어요.   FAP:REP:DS     하고/JKB→와/JKB


In [3]:
# 문장 단위로 오류 정보 집계
sentence_errors = err_rows.groupby(['표본 번호', '문장']).agg(
    error_signatures=('signature', list),
    correction_pairs=('correction_pair', list),
    error_locations=('오류 위치', list),
    error_patterns=('오류 양상', list),
    error_levels=('오류 층위', list),
).reset_index()

print(f'오류가 있는 문장 수: {len(sentence_errors):,}')
print()
print('=== 문장별 오류 집계 샘플 ===')
pd.set_option('display.max_colwidth', 120)
print(sentence_errors[['문장', 'error_signatures', 'correction_pairs']].head(10).to_string())

오류가 있는 문장 수: 65,791

=== 문장별 오류 집계 샘플 ===
                           문장                     error_signatures                       correction_pairs
0      그래서 열두 시에 다 끝난면 자도 돼요.  [CMAJ:REP:DC, CNNG:OM, FED:MIF:MCJ]  [그래서/MAJ→그리고/MAJ, ∅→전/NNG, 면/EC→면/EC]
1      수업이 끝난 후에 친구하고 약속 있어요.                 [FAP:REP:DS, FNP:OM]                [하고/JKB→와/JKB, ∅→이/JKS]
2  혼자 집에서 밥 먹고 텔레비전 보고 재미있어요.           [FOP:REP, FOP:OM, FED:REP]         [∅→을/JKO, ∅→을/JKO, 고/EC→아서/EC]
3   그래서 토요일 저녁에 친구하고 술을 마셨어요.                         [FAP:REP:DS]                         [하고/JKB→와/JKB]
4            그리고 이야가도 많이 했어요.                           [CNNG:MIF]                      [이야가/NNG→이야기/NNG]
5        식당에서 집에까지 40분쯤 걸렸어요.                            [FAP:ADD]                        [에/JKB→ADD/JKB]
6              저는 걸어서 집어 갔어요.                            [FAP:MIF]                          [어/JKB→에/JKB]
7             친구들을 이야기하고 싶어요.                            [FOP:REP]                          [을/JKO→과/JKB]
8   

In [6]:
# 오류 시그니처 빈도 분석
from collections import Counter

all_sigs = [sig for sigs in sentence_errors['error_signatures'] for sig in sigs]
sig_counts = Counter(all_sigs)

print(f'고유 오류 시그니처 수: {len(sig_counts):,}')
print()
print('=== 빈출 오류 시그니처 Top 20 ===')
for sig, count in sig_counts.most_common(20):
    print(f'  {count:>5}건  {sig}')

고유 오류 시그니처 수: 1,644

=== 빈출 오류 시그니처 Top 20 ===
  10555건  CNNG:MIF
   6424건  FAP:REP
   5975건  FNP:REP
   5358건  FED:REP
   4773건  CVV:REP
   4433건  CNNG:REP
   4072건  FOP:REP
   3976건  FXP:REP
   3622건  FOP:OM
   3558건  FPE:REP:ST
   3399건  FNP:OM
   2928건  FAP:OM
   2720건  FXP:OM
   2203건  CVV:MIF
   2060건  FFE:MIF:MCJ
   1684건  0:REP
   1621건  FAP:REP:DS
   1596건  CMAG:MIF
   1585건  FAE:REP
   1478건  FFE:REP


In [7]:
# 교정 쌍 빈도 분석
all_pairs = [p for ps in sentence_errors['correction_pairs'] for p in ps]
pair_counts = Counter(all_pairs)

print(f'고유 교정 쌍 수: {len(pair_counts):,}')
print()
print('=== 빈출 교정 쌍 Top 20 ===')
for pair, count in pair_counts.most_common(20):
    print(f'  {count:>5}건  {pair}')

고유 교정 쌍 수: 28,357

=== 빈출 교정 쌍 Top 20 ===
   2125건  ∅→을/JKO
   1847건  ∅→이/JKS
   1820건  이/JKS→을/JKO
   1812건  ∅→에/JKB
   1647건  ∅→는/JX
   1599건  ∅→를/JKO
   1557건  ∅→가/JKS
   1547건  ∅→았/EP
   1517건  ∅→었/EP
   1439건  을/JKO→이/JKS
   1409건  에/JKB→에서/JKB
   1154건  ∅→것/NNB
   1130건  이/JKS→은/JX
   1094건  에서/JKB→에/JKB
   1047건  ∅→의/JKG
    982건  에/JKB→ADD/JKB
    935건  가/JKS→는/JX
    932건  은/JX→이/JKS
    748건  ∅→은/JX
    743건  저/NP→나/NP


### Step 2. OpenSearch 인덱스 확장

기존 `korean_test` 인덱스에 오류 필드를 추가하고, 기존 문서(7,749건)에 오류 주석을 업데이트한다.

추가 필드:
- `error_signatures` (keyword[]) — `오류위치:오류양상:오류층위` 시그니처
- `correction_pairs` (keyword[]) — `원형태소/품사→교정형태소/품사` 쌍
- `error_patterns` (keyword[]) — 오류 양상 (REP, MIF, OM, ADD)
- `has_error` (boolean) — 오류 존재 여부

In [8]:
from opensearchpy import OpenSearch, helpers

client = OpenSearch(
    hosts=[{'host': '172.30.1.81', 'port': 9200}],
    http_auth=None,
    use_ssl=False,
    verify_certs=False,
)

INDEX_NAME = 'korean_test'

# 1) 새 필드 매핑 추가
new_mapping = {
    "properties": {
        "error_signatures": {"type": "keyword"},
        "correction_pairs": {"type": "keyword"},
        "error_patterns":   {"type": "keyword"},
        "has_error":        {"type": "boolean"},
    }
}

client.indices.put_mapping(index=INDEX_NAME, body=new_mapping)
print("매핑 추가 완료")
print(client.indices.get_mapping(index=INDEX_NAME)[INDEX_NAME]['mappings']['properties'].keys())

매핑 추가 완료
dict_keys(['correction_pairs', 'embedding', 'error_patterns', 'error_signatures', 'has_error', 'morphs', 'original_text'])


In [9]:
# 2) 기존 문서의 original_text를 모두 가져와서 말뭉치 오류 정보와 매칭
#    scroll API로 전체 문서 조회 (embedding 제외하여 속도 향상)

docs = []
resp = client.search(
    index=INDEX_NAME,
    body={"size": 1000, "query": {"match_all": {}}, "_source": ["original_text"]},
    scroll="2m",
)
scroll_id = resp['_scroll_id']
docs.extend(resp['hits']['hits'])

while len(resp['hits']['hits']) > 0:
    resp = client.scroll(scroll_id=scroll_id, scroll="2m")
    docs.extend(resp['hits']['hits'])

client.clear_scroll(scroll_id=scroll_id)
print(f"기존 문서 수: {len(docs)}")

# 문서 텍스트 → _id 매핑
doc_text_to_id = {d['_source']['original_text']: d['_id'] for d in docs}
print(f"고유 텍스트 수: {len(doc_text_to_id)}")

기존 문서 수: 7749
고유 텍스트 수: 7453


In [10]:
# 3) 말뭉치 오류 정보를 OpenSearch 문서에 매칭하여 bulk update 생성

# sentence_errors를 문장 텍스트로 검색할 수 있도록 딕셔너리화
error_by_text = {}
for _, row in sentence_errors.iterrows():
    text = row['문장']
    error_by_text[text] = {
        'error_signatures': row['error_signatures'],
        'correction_pairs': row['correction_pairs'],
        'error_patterns':   row['error_patterns'],
    }

# 매칭 통계
matched = 0
unmatched = 0
no_error = 0

actions = []
for text, doc_id in doc_text_to_id.items():
    if text in error_by_text:
        err = error_by_text[text]
        actions.append({
            "_op_type": "update",
            "_index": INDEX_NAME,
            "_id": doc_id,
            "doc": {
                "error_signatures": err['error_signatures'],
                "correction_pairs": err['correction_pairs'],
                "error_patterns":   err['error_patterns'],
                "has_error": True,
            }
        })
        matched += 1
    else:
        actions.append({
            "_op_type": "update",
            "_index": INDEX_NAME,
            "_id": doc_id,
            "doc": {
                "error_signatures": [],
                "correction_pairs": [],
                "error_patterns":   [],
                "has_error": False,
            }
        })
        no_error += 1

print(f"오류 매칭: {matched}건")
print(f"오류 없음: {no_error}건")
print(f"업데이트 대상: {len(actions)}건")

오류 매칭: 3725건
오류 없음: 3728건
업데이트 대상: 7453건


In [11]:
# 4) Bulk update 실행
success, errors = helpers.bulk(client, actions, raise_on_error=False)
print(f"성공: {success}건")
if errors:
    print(f"실패: {len(errors)}건")
    for e in errors[:5]:
        print(f"  {e}")

성공: 7453건


In [12]:
# 5) 업데이트 결과 검증 — 오류가 있는 문서 샘플 확인
resp = client.search(
    index=INDEX_NAME,
    body={
        "size": 5,
        "query": {"term": {"has_error": True}},
        "_source": ["original_text", "error_signatures", "correction_pairs", "error_patterns"],
    }
)

print(f"오류 문서 수: {client.count(index=INDEX_NAME, body={'query': {'term': {'has_error': True}}})['count']}")
print(f"정상 문서 수: {client.count(index=INDEX_NAME, body={'query': {'term': {'has_error': False}}})['count']}")
print()

for hit in resp['hits']['hits']:
    src = hit['_source']
    print(f"[{hit['_id']}] {src['original_text']}")
    print(f"  시그니처:  {src.get('error_signatures', [])}")
    print(f"  교정 쌍:   {src.get('correction_pairs', [])}")
    print(f"  오류 양상: {src.get('error_patterns', [])}")
    print()

오류 문서 수: 3725
정상 문서 수: 3728

[1478] 하지만 학교 숙제를 많아서 항상 피곤한다.
  시그니처:  ['FOP:REP', 'FPE:REP:ST', 'FFE:MIF:MCJ']
  교정 쌍:   ['를/JKO→가/JKS', '∅→았/EP', 'ㄴ다/EF→다/EF']
  오류 양상: ['REP', 'REP', 'MIF']

[1481] 행사 장소는 강북구청 2층 주민 복지과에서 있는다.
  시그니처:  ['CNNG:ADD', 'CVV:REP', 'FFE:MIF:MCJ']
  교정 쌍:   ['장소/NNG→ADD/NNG', '있/VV→하/VV', '는다/EF→ㄴ다/EF']
  오류 양상: ['ADD', 'REP', 'MIF']

[1483] 제가 좋아하는 방송은 드라마다.
  시그니처:  ['CNP:REP:SH']
  교정 쌍:   ['제/NP→나/NP']
  오류 양상: ['REP']

[1484] 하지만 점점 시간이 지나가지고 내용이 어려워졌기 때문에 의욕도 없어지고 처음보다는 공부를 안 하고 집에 가도 TV만 보고 있었다.
  시그니처:  ['FED;PE:ADD', 'CVX;PE:ADD', 'FED;PE:REP']
  교정 쌍:   ['아/EC→ADD/EC', '가지/VX→ADD/VX', '고/EC→면서/EC']
  오류 양상: ['ADD', 'ADD', 'REP']

[1507] 출러 후에 아루바이트 구한는 열심히 한다.
  시그니처:  ['CNNG:MIF', 'CNNG:MIF', 'FOP:OM', 'FAE:MIF;REP', 'FPE:REP:ST']
  교정 쌍:   ['출러/NNG→졸업/NNG', '아루바이트/NNG→아르바이트/NNG', '∅→를/JKO', '는/ETM→아서/EC', '∅→겠/EP']
  오류 양상: ['MIF', 'MIF', 'OM', 'MIF;REP', 'REP']



### Step 3. 검색 파이프라인 구현

**3-1**: 오류 시그니처 기반 term 검색 — 알려진 오류 유형으로 동일 패턴 검색  
**3-2**: 교정 쌍 기반 term 검색 — 구체적 교정 쌍으로 동일 오류 검색  
**3-3**: 2단계 검색 — BM25/벡터로 후보 확보 → 후보의 오류 주석에서 패턴 추출 → 오류 시그니처로 2차 검색

In [13]:
# 3-1: 오류 시그니처 기반 검색
def search_by_error_signature(signatures, size=10):
    """오류 시그니처로 동일 패턴의 문장을 검색"""
    query = {
        "size": size,
        "query": {
            "terms": {
                "error_signatures": signatures
            }
        },
        "_source": ["original_text", "error_signatures", "correction_pairs", "error_patterns"],
    }
    resp = client.search(index=INDEX_NAME, body=query)
    return resp['hits']['hits']

# 3-2: 교정 쌍 기반 검색
def search_by_correction_pair(pairs, size=10):
    """교정 쌍으로 동일 오류의 문장을 검색"""
    query = {
        "size": size,
        "query": {
            "terms": {
                "correction_pairs": pairs
            }
        },
        "_source": ["original_text", "error_signatures", "correction_pairs", "error_patterns"],
    }
    resp = client.search(index=INDEX_NAME, body=query)
    return resp['hits']['hits']

print("검색 함수 정의 완료")

검색 함수 정의 완료


In [14]:
# 3-1 테스트: "부사격 조사 대치 오류 (FAP:REP)" 로 검색
# 예) "친구하고→친구와" 같은 조사 혼동 오류를 가진 문장들
print("=== 3-1: 오류 시그니처 검색 [FAP:REP] ===")
print("(부사격 조사 위치에서 대치 오류가 발생한 문장)")
print()

hits = search_by_error_signature(["FAP:REP"], size=10)
for h in hits:
    src = h['_source']
    print(f"  [{h['_id']}] {src['original_text']}")
    print(f"    시그니처: {src['error_signatures']}")
    print(f"    교정 쌍:  {src['correction_pairs']}")
    print()

=== 3-1: 오류 시그니처 검색 [FAP:REP] ===
(부사격 조사 위치에서 대치 오류가 발생한 문장)

  [1542] 진실의 진겅하 아름다움이란 자신의 목표에 위해 영원히 노력하는 것이라고 저는 생각한다.
    시그니처: ['FGP:REP', 'CNNG:MIF', 'FAE:OM', 'FAP:REP', 'CNP:REP:SH']
    교정 쌍:  ['의/JKG→로/JKB', '진겅/NNG→진정/NNG', '∅→ㄴ/ETM', '에/JKB→를/JKO', '저/NP→나/NP']

  [1552] 친구가 캐나다의 대학교에서 공부하기로 했는데 나는 한국에 대학교에 입학하기로 했어.
    시그니처: ['FAP:REP', 'FFE:REP']
    교정 쌍:  ['에/JKB→의/JKG', '어/EF→다/EF']

  [1568] 한국에 다 보고 싶습니다.
    시그니처: ['FAP:REP']
    교정 쌍:  ['에/JKB→을/JKO']

  [1593] 제가 한국에 많이 좋아합니다.
    시그니처: ['FAP:REP']
    교정 쌍:  ['에/JKB→을/JKO']

  [1595] 한국에 온 후에 한국 문화에게 많이 알습니다.
    시그니처: ['FAP:REP', 'CVV:MIF:MCJ', 'FFE:MIF:MCJ']
    교정 쌍:  ['에게/JKB→를/JKO', '알/VV→알/VV', '습니다/EF→ㅂ니다/EF']

  [1614] 청소년은 미속하기 때문에 나쁜 버릇을 따라하곤 하는데 곱지 않은 말이 인터넷에서 잔뜩 차지한다면 이들에게 나쁜 영향을 미치기 마련이다.
    시그니처: ['CNNG:MIF', 'FAP:REP']
    교정 쌍:  ['미속/NNG→미숙/NNG', '에서/JKB→을/JKO']

  [1634] 한국에 생활이 좋아요.
    시그니처: ['FAP:REP']
    교정 쌍:  ['에/JKB→의/JKG']

  [1690] 착한 마음을 갖고 있는 사람들은 얼굴에 자연스럽게 다뜻한 모습을 볼 수 있다.
    시그니처: 

In [15]:
# 3-2 테스트: "이/JKS→을/JKO" (주격 조사를 목적격 조사로 써야 하는 오류) 로 검색
print("=== 3-2: 교정 쌍 검색 [이/JKS→을/JKO] ===")
print("(주격 '이'를 써야 할 곳에 목적격 '을'을 쓴 오류)")
print()

hits = search_by_correction_pair(["이/JKS→을/JKO"], size=10)
for h in hits:
    src = h['_source']
    print(f"  [{h['_id']}] {src['original_text']}")
    print(f"    시그니처: {src['error_signatures']}")
    print(f"    교정 쌍:  {src['correction_pairs']}")
    print()

=== 3-2: 교정 쌍 검색 [이/JKS→을/JKO] ===
(주격 '이'를 써야 할 곳에 목적격 '을'을 쓴 오류)

  [1763] 저는 여름하고 겨울이 싫어해요.
    시그니처: ['FJC:REP:DS', 'FNP:REP']
    교정 쌍:  ['하고/JC→과/JC', '이/JKS→을/JKO']

  [1811] 한국어를 배울 뿐만 아니라 시간이 재미있게 보낸다.
    시그니처: ['FNP:REP', 'FAE;PE:OM', 'CNNB;PE:OM', 'CVX;PE:OM']
    교정 쌍:  ['이/JKS→을/JKO', '∅→ㄹ/ETM', '∅→수/NNB', '∅→있/VX']

  [1996] 좋아하는 일이 할 수 있으면 일상생활도 재미있고 행복한다고 생각하기 때문입니다.
    시그니처: ['FNP:REP', 'FED:MIF:MCJ']
    교정 쌍:  ['이/JKS→을/JKO', 'ㄴ다고/EC→다고/EC']

  [1997] 왜냐하면 일본에서 고객님들을 도와줄 때 필요한 자격이나 면허를 취득했기 때문에 이런 생각이 나왔다.
    시그니처: ['CXSN:ADD:SH', 'FNP:REP', 'CVV:REP']
    교정 쌍:  ['님/XSN→ADD/XSN', '이/JKS→을/JKO', '나오/VV→하/VV']

  [2030] 청취자들의 관심이 있는 걸 보도하나?'
    시그니처: ['FGP:REP', 'FNP:REP', 'CVV:REP']
    교정 쌍:  ['의/JKG→이/JKS', '이/JKS→을/JKO', '있/VV→갖/VV']

  [2227] 한국에 오기 전에보다 지금이 한국말이 잘해요.
    시그니처: ['FAP:ADD', 'FNP:REP']
    교정 쌍:  ['에/JKB→ADD/JKB', '이/JKS→을/JKO']

  [2320] 좋아하는 음식이 요리를 할 수 있으면 좋겠습니다.
    시그니처: ['FNP:REP', 'FOP:ADD']
    교정 쌍:  ['이/JKS→을/JKO', '를/JKO→ADD/JKO']

 

In [16]:
# 3-3: 2단계 검색 (Two-Stage Retrieval)
# 임베딩 모델 로드 (BM25 + 벡터 1단계 검색용)
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./model/KURE-v1", local_files_only=True)
print(f"임베딩 모델 로드 완료 (dim={model.get_sentence_embedding_dimension()})")

/home/puju/py_playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:00<00:00, 403.25it/s, Materializing param=pooler.dense.weight]


임베딩 모델 로드 완료 (dim=1024)


In [17]:
from collections import Counter

def two_stage_search(query_text, stage1_k=20, stage2_size=10):
    """
    2단계 검색:
      Stage 1 — BM25 + 벡터 검색으로 후보 확보
      Stage 2 — 후보들의 오류 시그니처를 집계하여, 가장 빈번한 패턴으로 2차 검색
    """
    embedding = model.encode(query_text).tolist()
    
    # === Stage 1: BM25 + 벡터 하이브리드 검색 ===
    bm25_resp = client.search(index=INDEX_NAME, body={
        "size": stage1_k,
        "query": {"match": {"original_text": query_text}},
        "_source": ["original_text", "error_signatures", "correction_pairs", "has_error"],
    })
    
    vector_resp = client.search(index=INDEX_NAME, body={
        "size": stage1_k,
        "query": {"knn": {"embedding": {"vector": embedding, "k": stage1_k}}},
        "_source": ["original_text", "error_signatures", "correction_pairs", "has_error"],
    })
    
    # 후보 병합 (중복 제거)
    candidates = {}
    for hit in bm25_resp['hits']['hits'] + vector_resp['hits']['hits']:
        if hit['_id'] not in candidates:
            candidates[hit['_id']] = hit['_source']
    
    # === Stage 1 결과에서 오류 패턴 집계 ===
    sig_counter = Counter()
    pair_counter = Counter()
    error_candidates = 0
    
    for doc in candidates.values():
        if doc.get('has_error'):
            error_candidates += 1
            for sig in doc.get('error_signatures', []):
                sig_counter[sig] += 1
            for pair in doc.get('correction_pairs', []):
                pair_counter[pair] += 1
    
    print(f"=== Stage 1 결과 ===")
    print(f"  입력: \"{query_text}\"")
    print(f"  후보 수: {len(candidates)}건 (오류 문장: {error_candidates}건)")
    print(f"  빈출 시그니처: {sig_counter.most_common(5)}")
    print(f"  빈출 교정 쌍: {pair_counter.most_common(5)}")
    
    if not sig_counter:
        print("  → 후보 중 오류 문장이 없음. Stage 2 생략.")
        return []
    
    # === Stage 2: 가장 빈번한 시그니처로 2차 검색 ===
    top_sigs = [sig for sig, _ in sig_counter.most_common(3)]
    
    stage2_resp = client.search(index=INDEX_NAME, body={
        "size": stage2_size,
        "query": {
            "bool": {
                "must": [{"terms": {"error_signatures": top_sigs}}],
                "must_not": [{"ids": {"values": list(candidates.keys())}}],  # Stage 1 결과 제외
            }
        },
        "_source": ["original_text", "error_signatures", "correction_pairs"],
    })
    
    print(f"\n=== Stage 2 결과 (시그니처: {top_sigs}) ===")
    results = stage2_resp['hits']['hits']
    for h in results:
        src = h['_source']
        print(f"  [{h['_id']}] {src['original_text']}")
        print(f"    시그니처: {src['error_signatures']}")
        print(f"    교정 쌍:  {src['correction_pairs']}")
        print()
    
    if not results:
        print("  → Stage 1 결과 제외 후 추가 결과 없음.")
    
    return results

print("2단계 검색 함수 정의 완료")

2단계 검색 함수 정의 완료


In [18]:
# 3-3 테스트: 가설 2에서 사용한 테스트 문장들로 2단계 검색
print("=" * 80)
print("테스트 1: 오타 오류 (뱡완 → 병원)")
print("=" * 80)
two_stage_search("저는 어제 뱡완에 갔어요.")

테스트 1: 오타 오류 (뱡완 → 병원)
=== Stage 1 결과 ===
  입력: "저는 어제 뱡완에 갔어요."
  후보 수: 36건 (오류 문장: 16건)
  빈출 시그니처: [('FAP:OM', 4), ('CNNG:MIF', 4), ('FAP:REP:DS', 4), ('FAP:REP', 2), ('FOP:REP', 2)]
  빈출 교정 쌍: [('∅→에/JKB', 4), ('하고/JKB→와/JKB', 4), ('을/JKO→과/JKB', 2), ('에/JKB→의/JKG', 1), ('비군/NNG→피곤/NNG', 1)]

=== Stage 2 결과 (시그니처: ['FAP:OM', 'CNNG:MIF', 'FAP:REP:DS']) ===
  [1507] 출러 후에 아루바이트 구한는 열심히 한다.
    시그니처: ['CNNG:MIF', 'CNNG:MIF', 'FOP:OM', 'FAE:MIF;REP', 'FPE:REP:ST']
    교정 쌍:  ['출러/NNG→졸업/NNG', '아루바이트/NNG→아르바이트/NNG', '∅→를/JKO', '는/ETM→아서/EC', '∅→겠/EP']

  [1508] 화사 근처에 있는 아파트를 살고 싶다.
    시그니처: ['CNNG:MIF', 'FOP:REP']
    교정 쌍:  ['화사/NNG→회사/NNG', '를/JKO→에서/JKB']

  [1517] 한국에서 차 타기가 좀 피곱니다.
    시그니처: ['CNNG:MIF', 'CVC:REP']
    교정 쌍:  ['피고/NNG→피곤/NNG', '이/VCP→하/XSA']

  [1519] 학교에 제육과 있는데, 수영하야 됩니다.
    시그니처: ['CNNG:MIF', 'CNNG:REP', 'FNP:OM', 'CXSV:MIF:MCJ', 'FED:MIF:MCJ']
    교정 쌍:  ['제육과/NNG→체육/NNG', '∅→수업/NNG', '∅→이/JKS', '하/XSV→하/XSV', '야/EC→아야/EC']

  [1521] 한국 음식 중에서 제일 기치를 좋아해요.
  

[{'_index': 'korean_test',
  '_id': '1507',
  '_score': 1.0,
  '_source': {'correction_pairs': ['출러/NNG→졸업/NNG',
    '아루바이트/NNG→아르바이트/NNG',
    '∅→를/JKO',
    '는/ETM→아서/EC',
    '∅→겠/EP'],
   'error_signatures': ['CNNG:MIF',
    'CNNG:MIF',
    'FOP:OM',
    'FAE:MIF;REP',
    'FPE:REP:ST'],
   'original_text': '출러 후에 아루바이트 구한는 열심히 한다.'}},
 {'_index': 'korean_test',
  '_id': '1508',
  '_score': 1.0,
  '_source': {'correction_pairs': ['화사/NNG→회사/NNG', '를/JKO→에서/JKB'],
   'error_signatures': ['CNNG:MIF', 'FOP:REP'],
   'original_text': '화사 근처에 있는 아파트를 살고 싶다.'}},
 {'_index': 'korean_test',
  '_id': '1517',
  '_score': 1.0,
  '_source': {'correction_pairs': ['피고/NNG→피곤/NNG', '이/VCP→하/XSA'],
   'error_signatures': ['CNNG:MIF', 'CVC:REP'],
   'original_text': '한국에서 차 타기가 좀 피곱니다.'}},
 {'_index': 'korean_test',
  '_id': '1519',
  '_score': 1.0,
  '_source': {'correction_pairs': ['제육과/NNG→체육/NNG',
    '∅→수업/NNG',
    '∅→이/JKS',
    '하/XSV→하/XSV',
    '야/EC→아야/EC'],
   'error_signatures': ['CNNG

In [19]:
print("=" * 80)
print("테스트 2: 조사 오류 (친구를 같이 → 친구와 같이)")
print("=" * 80)
two_stage_search("저는 친구를 같이 영화를 봤어요.")

테스트 2: 조사 오류 (친구를 같이 → 친구와 같이)
=== Stage 1 결과 ===
  입력: "저는 친구를 같이 영화를 봤어요."
  후보 수: 36건 (오류 문장: 21건)
  빈출 시그니처: [('FAP:REP:DS', 8), ('CNNG:MIF', 3), ('FOP:REP', 3), ('FNP:REP', 2), ('FED:REP', 2)]
  빈출 교정 쌍: [('하고/JKB→와/JKB', 6), ('를/JKO→가/JKS', 3), ('가/JKS→는/JX', 2), ('∅→과/JKB', 1), ('하고/JC→ADD/JC', 1)]

=== Stage 2 결과 (시그니처: ['FAP:REP:DS', 'CNNG:MIF', 'FOP:REP']) ===
  [1478] 하지만 학교 숙제를 많아서 항상 피곤한다.
    시그니처: ['FOP:REP', 'FPE:REP:ST', 'FFE:MIF:MCJ']
    교정 쌍:  ['를/JKO→가/JKS', '∅→았/EP', 'ㄴ다/EF→다/EF']

  [1507] 출러 후에 아루바이트 구한는 열심히 한다.
    시그니처: ['CNNG:MIF', 'CNNG:MIF', 'FOP:OM', 'FAE:MIF;REP', 'FPE:REP:ST']
    교정 쌍:  ['출러/NNG→졸업/NNG', '아루바이트/NNG→아르바이트/NNG', '∅→를/JKO', '는/ETM→아서/EC', '∅→겠/EP']

  [1508] 화사 근처에 있는 아파트를 살고 싶다.
    시그니처: ['CNNG:MIF', 'FOP:REP']
    교정 쌍:  ['화사/NNG→회사/NNG', '를/JKO→에서/JKB']

  [1511] 한국 음악을 아주 유행합니다.
    시그니처: ['FOP:REP']
    교정 쌍:  ['을/JKO→이/JKS']

  [1517] 한국에서 차 타기가 좀 피곱니다.
    시그니처: ['CNNG:MIF', 'CVC:REP']
    교정 쌍:  ['피고/NNG→피곤/NNG', '이/VCP→하/XSA']

 

[{'_index': 'korean_test',
  '_id': '1478',
  '_score': 1.0,
  '_source': {'correction_pairs': ['를/JKO→가/JKS', '∅→았/EP', 'ㄴ다/EF→다/EF'],
   'error_signatures': ['FOP:REP', 'FPE:REP:ST', 'FFE:MIF:MCJ'],
   'original_text': '하지만 학교 숙제를 많아서 항상 피곤한다.'}},
 {'_index': 'korean_test',
  '_id': '1507',
  '_score': 1.0,
  '_source': {'correction_pairs': ['출러/NNG→졸업/NNG',
    '아루바이트/NNG→아르바이트/NNG',
    '∅→를/JKO',
    '는/ETM→아서/EC',
    '∅→겠/EP'],
   'error_signatures': ['CNNG:MIF',
    'CNNG:MIF',
    'FOP:OM',
    'FAE:MIF;REP',
    'FPE:REP:ST'],
   'original_text': '출러 후에 아루바이트 구한는 열심히 한다.'}},
 {'_index': 'korean_test',
  '_id': '1508',
  '_score': 1.0,
  '_source': {'correction_pairs': ['화사/NNG→회사/NNG', '를/JKO→에서/JKB'],
   'error_signatures': ['CNNG:MIF', 'FOP:REP'],
   'original_text': '화사 근처에 있는 아파트를 살고 싶다.'}},
 {'_index': 'korean_test',
  '_id': '1511',
  '_score': 1.0,
  '_source': {'correction_pairs': ['을/JKO→이/JKS'],
   'error_signatures': ['FOP:REP'],
   'original_text': '한국 음악을 아주 유행합니

In [20]:
print("=" * 80)
print("테스트 3: 어미 오류 (힘든입니다 → 힘듭니다)")
print("=" * 80)
two_stage_search("공부가 힘든입니다.")

테스트 3: 어미 오류 (힘든입니다 → 힘듭니다)
=== Stage 1 결과 ===
  입력: "공부가 힘든입니다."
  후보 수: 38건 (오류 문장: 10건)
  빈출 시그니처: [('FPE:REP:ST', 2), ('FAE:REP', 1), ('CNNG;PE:MIF', 1), ('CMAG:ADD', 1), ('FFE;PE:REP', 1)]
  빈출 교정 쌍: [('∅→었/EP', 2), ('ㄴ/ETM→ㅁ/ETN', 1), ('데/NNG→때/NNG', 1), ('얼마나/MAG→ADD/MAG', 1), ('에요/EF→ㄴ/ETM', 1)]

=== Stage 2 결과 (시그니처: ['FPE:REP:ST', 'FAE:REP', 'CNNG;PE:MIF']) ===
  [1478] 하지만 학교 숙제를 많아서 항상 피곤한다.
    시그니처: ['FOP:REP', 'FPE:REP:ST', 'FFE:MIF:MCJ']
    교정 쌍:  ['를/JKO→가/JKS', '∅→았/EP', 'ㄴ다/EF→다/EF']

  [1507] 출러 후에 아루바이트 구한는 열심히 한다.
    시그니처: ['CNNG:MIF', 'CNNG:MIF', 'FOP:OM', 'FAE:MIF;REP', 'FPE:REP:ST']
    교정 쌍:  ['출러/NNG→졸업/NNG', '아루바이트/NNG→아르바이트/NNG', '∅→를/JKO', '는/ETM→아서/EC', '∅→겠/EP']

  [1708] 힘드는 것도 많이 있지만 노력하면 꼭 꿈은 이루어지는 것을 믿고 싶다.
    시그니처: ['FAE;PE:MIF:MCJ', 'FAE:REP']
    교정 쌍:  ['는/ETM→ㄴ/ETM', '는/ETM→ㄴ다는/ETM']

  [1807] 최근 사람들이 쉬는 시간에 제일 좋아하는 취미 활동이 조사가 있는데 조사 결과는 집에서 텔레비정에 방송을 보는 것이 제일 많아다.
    시그니처: ['FNP:REP', '0:REP', '0:REP', 'FPE:REP:ST', 'CNNG:MIF', 'FAP:ADD', '

[{'_index': 'korean_test',
  '_id': '1478',
  '_score': 1.0,
  '_source': {'correction_pairs': ['를/JKO→가/JKS', '∅→았/EP', 'ㄴ다/EF→다/EF'],
   'error_signatures': ['FOP:REP', 'FPE:REP:ST', 'FFE:MIF:MCJ'],
   'original_text': '하지만 학교 숙제를 많아서 항상 피곤한다.'}},
 {'_index': 'korean_test',
  '_id': '1507',
  '_score': 1.0,
  '_source': {'correction_pairs': ['출러/NNG→졸업/NNG',
    '아루바이트/NNG→아르바이트/NNG',
    '∅→를/JKO',
    '는/ETM→아서/EC',
    '∅→겠/EP'],
   'error_signatures': ['CNNG:MIF',
    'CNNG:MIF',
    'FOP:OM',
    'FAE:MIF;REP',
    'FPE:REP:ST'],
   'original_text': '출러 후에 아루바이트 구한는 열심히 한다.'}},
 {'_index': 'korean_test',
  '_id': '1708',
  '_score': 1.0,
  '_source': {'correction_pairs': ['는/ETM→ㄴ/ETM', '는/ETM→ㄴ다는/ETM'],
   'error_signatures': ['FAE;PE:MIF:MCJ', 'FAE:REP'],
   'original_text': '힘드는 것도 많이 있지만 노력하면 꼭 꿈은 이루어지는 것을 믿고 싶다.'}},
 {'_index': 'korean_test',
  '_id': '1807',
  '_score': 1.0,
  '_source': {'correction_pairs': ['이/JKS→에/JKB',
    '∅→대하/VV',
    '∅→ㄴ/ETM',
    '∅→었/EP',
    '

In [21]:
print("=" * 80)
print("테스트 4: 복합 오류")
print("=" * 80)
two_stage_search("내 월꺼 아네 있는데요")

테스트 4: 복합 오류
=== Stage 1 결과 ===
  입력: "내 월꺼 아네 있는데요"
  후보 수: 38건 (오류 문장: 11건)
  빈출 시그니처: [('CVV:REP', 3), ('CNP:ADD', 2), ('FED:REP', 1), ('FFE:REP:SH', 1), ('FFE:REP', 1)]
  빈출 교정 쌍: [('나/NP→ADD/NP', 2), ('면서/EC→니/EC', 1), ('있/VV→되/VV', 1), ('아/EF→다/EF', 1), ('구하/VV→돕/VV', 1)]

=== Stage 2 결과 (시그니처: ['CVV:REP', 'CNP:ADD', 'FED:REP']) ===
  [1481] 행사 장소는 강북구청 2층 주민 복지과에서 있는다.
    시그니처: ['CNNG:ADD', 'CVV:REP', 'FFE:MIF:MCJ']
    교정 쌍:  ['장소/NNG→ADD/NNG', '있/VV→하/VV', '는다/EF→ㄴ다/EF']

  [1550] 한국에서 정통 곳도 있고 미래처럼 곳도 있으니까 너무 좋아한다.
    시그니처: ['FED:REP']
    교정 쌍:  ['으니까/EC→어서/EC']

  [1555] 학원 친구들 덕분에 한국말도 빨리 늘었고 한국 생활도 빨리 익숙해졌고 외롭지 않고 어려움을 잘 견디고 있습니다.
    시그니처: ['FED:REP']
    교정 쌍:  ['고/EC→게/EC']

  [1616] 책 사기에는 어려운 일이 많아서 한국에서는 인터넷을 이용해서 언어에 대한 많는 정보를 받게 되고 도움이 된다.
    시그니처: ['CNNG:REP', 'FAE:MIF:MCJ', 'FED:REP']
    교정 쌍:  ['일/NNG→점/NNG', '는/ETM→은/ETM', '고/EC→어서/EC']

  [1829] 정부가 사람들이 나우르즈의 전통적인 분위기를 느끼라고 각 도시에서 유르타라고 하는 전통적인 집 백 개 이상 세우고 여러 행사를 열린다.
    시그니처: ['FED:REP', 'FOP:OM', 'CV

[{'_index': 'korean_test',
  '_id': '1481',
  '_score': 1.0,
  '_source': {'correction_pairs': ['장소/NNG→ADD/NNG',
    '있/VV→하/VV',
    '는다/EF→ㄴ다/EF'],
   'error_signatures': ['CNNG:ADD', 'CVV:REP', 'FFE:MIF:MCJ'],
   'original_text': '행사 장소는 강북구청 2층 주민 복지과에서 있는다.'}},
 {'_index': 'korean_test',
  '_id': '1550',
  '_score': 1.0,
  '_source': {'correction_pairs': ['으니까/EC→어서/EC'],
   'error_signatures': ['FED:REP'],
   'original_text': '한국에서 정통 곳도 있고 미래처럼 곳도 있으니까 너무 좋아한다.'}},
 {'_index': 'korean_test',
  '_id': '1555',
  '_score': 1.0,
  '_source': {'correction_pairs': ['고/EC→게/EC'],
   'error_signatures': ['FED:REP'],
   'original_text': '학원 친구들 덕분에 한국말도 빨리 늘었고 한국 생활도 빨리 익숙해졌고 외롭지 않고 어려움을 잘 견디고 있습니다.'}},
 {'_index': 'korean_test',
  '_id': '1616',
  '_score': 1.0,
  '_source': {'correction_pairs': ['일/NNG→점/NNG', '는/ETM→은/ETM', '고/EC→어서/EC'],
   'error_signatures': ['CNNG:REP', 'FAE:MIF:MCJ', 'FED:REP'],
   'original_text': '책 사기에는 어려운 일이 많아서 한국에서는 인터넷을 이용해서 언어에 대한 많는 정보를 받게 되고 도움이 된다.'}

---
## 실험 결과 및 분석

### 3-1 / 3-2: 직접 검색 (오류 시그니처 / 교정 쌍)

오류 유형을 **미리 알고 있는 경우**, 직접 검색의 정밀도가 매우 높다.

| 검색 방식 | 검색 조건 | 결과 | 평가 |
|-----------|----------|------|------|
| 3-1 시그니처 | `FAP:REP` (부사격 조사 대치) | 10건 모두 부사격 조사 오류 문장 | **정밀도 100%** |
| 3-2 교정 쌍 | `이/JKS→을/JKO` (주격→목적격) | 10건 모두 해당 교정 쌍 포함 | **정밀도 100%** |

→ 오류 주석 데이터를 직접 검색 조건으로 활용하면, 동일/유사 오류 패턴의 문장을 정확하게 검색할 수 있음.

### 3-3: 2단계 검색 (BM25/벡터 → 패턴 역추정 → 시그니처 2차 검색)

| 테스트 | 입력 문장 | 기대 오류 유형 | Stage 1 추정 시그니처 | Stage 2 적합도 | 판정 |
|--------|----------|--------------|---------------------|---------------|------|
| 1 | "저는 어제 **뱡완**에 갔어요." | 오타 (뱡완→병원) | `FAP:OM`, `CNNG:MIF`, `FAP:REP:DS` | CNNG:MIF 관련 결과 다수 (화사→회사 등) | **부분 성공** — 오타(MIF) 패턴은 잡았으나 '뱡완' 자체를 매칭하지 못함 |
| 2 | "저는 친구**를** 같이 영화를 봤어요." | 조사 대치 (를→와) | `FAP:REP:DS`, `CNNG:MIF`, `FOP:REP` | FAP:REP:DS 관련 결과 포함 (하고→와 등) | **부분 성공** — 조사 오류 패턴은 나오지만 정확한 '를→와' 매칭은 아님 |
| 3 | "공부가 **힘든입니다**." | 어미 오류 (힘든입니다→힘듭니다) | `FPE:REP:ST`, `FAE:REP`, `CNNG;PE:MIF` | 시제/문체 관련 오류 결과 혼재 | **실패** — 어미 결합 오류를 정확히 추정하지 못함 |
| 4 | "내 **월꺼** **아네** 있는데요" | 복합 오류 (월꺼→윗것, 아네→안에) | `CVV:REP`, `CNP:ADD`, `FED:REP` | 다양한 오류 유형 혼재, 입력과 무관한 결과 | **실패** — Stage 1 후보의 노이즈가 심하여 역추정 무의미 |

### 핵심 발견

1. **오류 주석 데이터의 검색 가치는 입증됨**: 3-1/3-2에서 시그니처/교정 쌍을 알면 정밀도 100%의 검색이 가능
2. **병목은 "입력 문장의 오류 유형 식별"**: 사용자가 입력한 문장의 오류가 무엇인지 모르는 상태에서 말뭉치 후보로부터 역추정하는 방식은 노이즈에 취약

### 한계: 2단계 검색의 역추정 과정

- **오타/철자 오류**: BM25가 오타 단어(뱡완, 월꺼)를 매칭하지 못함 → Stage 1 후보 품질 낮음 → 패턴 추정 실패
- **조사 오류**: 유사 문장에 조사 오류가 흔하여 Stage 1에서 비교적 합리적 추정 가능하나, 정확한 교정 쌍 특정은 어려움
- **어미/복합 오류**: Stage 1 후보가 다양한 오류를 포함하여 빈출 시그니처가 입력 오류와 무관한 경우 많음

### 다음 단계: 가설 4

> **LLM 기반 오류 진단 → 구조 검색**: LLM이 입력 문장의 오류를 직접 진단하고, 오류 시그니처/교정 쌍을 생성한 뒤, 3-1/3-2 방식으로 직접 검색

- 역추정(Stage 1 → 패턴 추출)의 병목을 LLM으로 대체
- LLM이 "뱡완→병원"을 직접 인식 → `CNNG:MIF` 시그니처와 `뱡완/NNG→병원/NNG` 교정 쌍 생성 → 정밀 검색
- 가설 4에서 구현 및 검증 예정